In [1]:
import pandas as pd
from datetime import datetime
from some_func import get_time_of_day, holidays
from collections import Counter


data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Преобразуем временную метку в объект datetime
data_train['timestamp'] = pd.to_datetime(data_train['timestamp'])

#Создаем признаки день недели, время суток, праздничный ли день
data_train['day_of_the_week'] = data_train['timestamp'].dt.dayofweek
data_train['part_of_the_day'] = data_train['timestamp'].dt.hour.apply(get_time_of_day)
data_train['holiday'] = data_train['timestamp'].dt.date.astype(str).isin(holidays)

# Создаем признак время года
data_train['season'] = pd.cut(data_train['timestamp'].dt.month,
                        bins=[0, 3, 6, 9, 12],
                        labels=['winter', 'spring', 'summer', 'autumn'])

# Создаем признак времени между проходами юзера через турникет
data_train['time_between_passes'] = data_train.groupby('gate_id')['timestamp'].diff().dt.total_seconds()
data_train['time_between_passes'] = data_train['time_between_passes'].fillna(0)

user_ids = list(Counter(data_train.user_id).keys())
data_train['Event'] = ''
data_train['num_people'] = 1
for idx in user_ids:
    user_stamps = data_train.loc[(data_train['user_id'] == idx)].sort_values(by=['timestamp'])
    im = 0
    for i, stamp in user_stamps.iterrows():
        print(i)
        if im == 0:
            stamp['Event'] = 'in'
            im += 1
        else:
            if user_stamps.loc[i-1, 'Event'] == 'in':
                stamp['Event'] = 'out'
            else:
                stamp['Event'] = 'in'

data_train

In [2]:
user_stamps

In [3]:
import pandas as pd
from some_func import get_time_of_day, holidays

data_train = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

# Преобразуем временную метку в объект datetime
data_train['timestamp'] = pd.to_datetime(data_train['timestamp'])

#Создаем признаки день недели, время суток, праздничный ли день
data_train['day_of_the_week'] = data_train['timestamp'].dt.dayofweek
data_train['part_of_the_day'] = data_train['timestamp'].dt.hour.apply(get_time_of_day)
data_train['holiday'] = data_train['timestamp'].dt.date.astype(str).isin(holidays)

# Создаем признак время года
data_train['season'] = pd.cut(data_train['timestamp'].dt.month,
                        bins=[0, 3, 6, 9, 12],
                        labels=['winter', 'spring', 'summer', 'autumn'])

# Создаем признак времени между проходами юзера через турникет
#data_train['time_between_passes'] = data_train.groupby('user_id')['timestamp'].diff().dt.total_seconds()
#data_train['time_between_passes'] = data_train['time_between_passes'].fillna(0)


data_train['day_of_month'] = data_train['timestamp'].dt.day
data_train['hour'] = data_train['timestamp'].dt.hour
data_train['min'] = data_train['timestamp'].dt.minute
data_train['sec'] = data_train['timestamp'].dt.second

data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'gate_id'], prefix='gate_id'))
data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'day_of_the_week'], prefix='day_of_week'))
data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'part_of_the_day'], prefix='part_of_day'))
data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'season'], prefix='season'))
data_train['holiday'] = data_train['holiday'].astype(int)

data_train['time_between_passes'] = data_train.groupby('gate_id')['timestamp'].diff().dt.total_seconds()
data_train['time_between_passes'] = data_train['time_between_passes'].fillna(0)
data_train['delta'] = data_train['timestamp'].diff().dt.total_seconds()
data_train['delta'] = data_train['delta'].fillna(0)

data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'hour'], prefix='hour'))
data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'min'], prefix='min'))
data_train = data_train.join(pd.get_dummies(data_train.loc[:, 'day_of_month'], prefix='day_of_month'))


data_train = data_train.drop(columns=['timestamp', 'gate_id', 'day_of_the_week', 'part_of_the_day', 'season'])

data_train.to_csv('5_clear_train.csv', index=False)

,row_id,user_id,holiday,day_of_month,hour,min,sec,gate_id_-1,gate_id_0,gate_id_1,...,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31
0,0,18,0,29,9,8,54,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,18,0,29,9,9,54,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,18,0,29,9,9,54,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,18,0,29,9,10,6,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,18,0,29,9,10,8,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37513,37513,6,1,31,20,38,56,0,0,0,...,0,0,0,0,0,0,0,0,0,1
37514,37514,6,1,31,20,39,22,0,0,0,...,0,0,0,0,0,0,0,0,0,1
37515,37515,6,1,31,20,39,23,0,0,0,...,0,0,0,0,0,0,0,0,0,1
37516,37516,6,1,31,20,39,31,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
data_train

In [6]:
import pandas as pd
from datetime import datetime
from some_func import get_time_of_day, holidays
from collections import Counter

data_test = pd.read_csv('test.csv')

# Преобразуем временную метку в объект datetime
data_test['timestamp'] = pd.to_datetime(data_test['timestamp'])

# Создаем признаки день недели, время суток, праздничный ли день
data_test['day_of_the_week'] = data_test['timestamp'].dt.dayofweek
data_test['part_of_the_day'] = data_test['timestamp'].dt.hour.apply(get_time_of_day)
data_test['holiday'] = data_test['timestamp'].dt.date.astype(str).isin(holidays)

# Создаем признак время года
data_test['season'] = pd.cut(data_test['timestamp'].dt.month,
                             bins=[0, 3, 6, 9, 12],
                             labels=['winter', 'spring', 'summer', 'autumn'])

# Создаем признак времени между проходами юзера через турникет
#data_test['time_between_passes'] = data_test.groupby('user_id')['timestamp'].diff().dt.total_seconds()
#data_test['time_between_passes'] = data_test['time_between_passes'].fillna(0)


data_test['day_of_month'] = data_test['timestamp'].dt.day
data_test['hour'] = data_test['timestamp'].dt.hour
data_test['min'] = data_test['timestamp'].dt.minute
data_test['sec'] = data_test['timestamp'].dt.second

data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'gate_id'], prefix='gate_id'))
data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'day_of_the_week'], prefix='day_of_week'))
data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'part_of_the_day'], prefix='part_of_day'))
data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'season'], prefix='season'))
data_test['holiday'] = data_test['holiday'].astype(int)

data_test['time_between_passes'] = data_test.groupby('gate_id')['timestamp'].diff().dt.total_seconds()
data_test['time_between_passes'] = data_test['time_between_passes'].fillna(0)
data_test['delta'] = data_test['timestamp'].diff().dt.total_seconds()
data_test['delta'] = data_test['delta'].fillna(0)

data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'hour'], prefix='hour'))
data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'min'], prefix='min'))
data_test = data_test.join(pd.get_dummies(data_test.loc[:, 'day_of_month'], prefix='day_of_month'))

data_test = data_test.drop(columns=['timestamp', 'gate_id', 'day_of_the_week', 'part_of_the_day', 'season'])

data_test.to_csv('5_clear_test.csv', index=False)


In [8]:
data_train

,row_id,holiday,day_of_month,hour,min,sec,gate_id_-1,gate_id_1,gate_id_2,gate_id_3,...,day_of_month_22,day_of_month_23,day_of_month_24,day_of_month_25,day_of_month_26,day_of_month_27,day_of_month_28,day_of_month_29,day_of_month_30,day_of_month_31
0,37518,1,3,8,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,37519,1,3,8,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37520,1,3,8,21,18,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37521,1,3,8,21,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37522,1,3,8,21,39,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7120,44638,1,24,19,43,36,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7121,44639,1,24,19,44,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7122,44640,1,24,19,44,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7123,44641,1,24,19,44,9,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [51]:
data_test

,row_id,holiday,day_of_month,hour,min,sec,gate_id_-1,gate_id_1,gate_id_2,gate_id_3,...,min_50,min_51,min_52,min_53,min_54,min_55,min_56,min_57,min_58,min_59
0,37518,0,3,8,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,37519,0,3,8,21,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,37520,0,3,8,21,18,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,37521,0,3,8,21,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,37522,0,3,8,21,39,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7120,44638,0,24,19,43,36,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7121,44639,0,24,19,44,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7122,44640,0,24,19,44,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7123,44641,0,24,19,44,9,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
import configparser
import os
import datetime
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm
import neptune
from data_functions import TurnstileDataset, get_loaders
from metrics_functions import compute_accuracy, validation_loss
import numpy as np

if torch.cuda.is_available():
    dev = 'cuda:0'
else:
    dev = 'cpu'
device = torch.device(dev)

model = nn.Sequential(
    nn.Linear(110, 128),
    #nn.BatchNorm1d(128),
    nn.ReLU(),
    nn.Dropout(0.069),
    nn.Linear(128, 139),
    nn.ReLU(),
    nn.Dropout(0.0155),
    nn.Linear(139, 195),
    nn.ReLU(),
    nn.Dropout(0.017),
    nn.Linear(195, 58),
)
model.to(device)
model.type(torch.cuda.FloatTensor)

model.load_state_dict( torch.load(os.path.join('optimized/optims.pth'), map_location=torch.device("cuda")) )


test_set = TurnstileDataset(test=True, normalize=True)
test_loader = DataLoader(test_set, batch_size=1)


model.eval()

predictions = model(torch.Tensor(test_set.X_test).to(device))

predictions_labels = torch.argmax(predictions, dim=1)
predictions_labels_list = list(np.array(predictions_labels.cpu()))

output = pd.DataFrame({'row_id' : list(range(37518, 44643)),
                       'target' : predictions_labels_list}, columns=['row_id', 'target'])
output.to_csv(os.path.join('outputs/output__' + datetime.datetime.now().strftime('%d%m%y%H%M') + '.csv'), index=False)

ValueError: 2 columns passed, passed data had 7125 columns

In [58]:
torch.argmax(predictions, dim=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [60]:
predictions = model(torch.Tensor(test_set.X_test).to(device))
predictions

tensor([[ -5.7816,  11.1053, -10.2409,  ...,   3.6145,  -8.1807, -15.2887],
        [ -5.7816,  11.1053, -10.2409,  ...,   3.6145,  -8.1807, -15.2887],
        [ -9.5426,   8.4816, -13.6751,  ...,   1.1495,  -5.2589, -11.4211],
        ...,
        [ -8.3594,   0.8504,  -6.3661,  ...,  -9.5923,  -8.4599,  -5.5422],
        [ -7.8520,  -7.0630, -12.2311,  ...,  -7.9301, -10.7578,   2.0349],
        [ -7.8520,  -7.0630, -12.2311,  ...,  -7.9301, -10.7578,   2.0349]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [78]:
predictions_labels = torch.argmax(predictions, dim=1)
predictions_labelss = list(np.array(predictions_labels.cpu()))

In [84]:
pd.DataFrame({'row_id' : list(range(37518, 44643)),
                       'target' : predictions_labelss}, columns=['row_id', 'target']).to_csv(os.path.join('outputs/output__' + datetime.datetime.now().strftime('%d%m%y%H%M') + '.csv'), index=False)

In [81]:
len(list(range(37518, 7126)))

0